# Predict developer salary, using stackoverflow developer survey 2018 dataset

# 1. Business Understanding
### I want to investigate the annual income of developers based on stackoverflow survey 2018 data for a project of data science course taken at udacity.

## Questions I have for this dataset
### Main Question: What is the best way to predict salary using this dataset and machine learning
Q1. What is the best feature to predict the salary
   
Q2. If I collect a few features that predict best and re-train, will the result be better?    
    
Q3. What would be the result if we used all the features?

# 2. Data Understanding

## Import Data

In [5]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
pd.set_option("display.max_columns", None)
path = 'survey_results_public_2017.csv'
df = pd.read_csv(path)
print(df.shape)
df.head(3)

(51392, 154)


,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,InvestTimeTools,WorkPayCare,KinshipDevelopers,ChallengeMyself,CompetePeers,ChangeWorld,JobSeekingStatus,HoursPerWeek,LastNewJob,AssessJobIndustry,AssessJobRole,AssessJobExp,AssessJobDept,AssessJobTech,AssessJobProjects,AssessJobCompensation,AssessJobOffice,AssessJobCommute,AssessJobRemote,AssessJobLeaders,AssessJobProfDevel,AssessJobDiversity,AssessJobProduct,AssessJobFinances,ImportantBenefits,ClickyKeys,JobProfile,ResumePrompted,LearnedHiring,ImportantHiringAlgorithms,ImportantHiringTechExp,ImportantHiringCommunication,ImportantHiringOpenSource,ImportantHiringPMExp,ImportantHiringCompanies,ImportantHiringTitles,ImportantHiringEducation,ImportantHiringRep,ImportantHiringGettingThingsDone,Currency,Overpaid,TabsSpaces,EducationImportant,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,CousinEducation,WorkStart,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform,IDE,AuditoryEnvironment,Methodology,VersionControl,CheckInCode,ShipIt,OtherPeoplesCode,ProjectManagement,EnjoyDebugging,InTheZone,DifficultCommunication,CollaborateRemote,MetricAssess,EquipmentSatisfiedMonitors,EquipmentSatisfiedCPU,EquipmentSatisfiedRAM,EquipmentSatisfiedStorage,EquipmentSatisfiedRW,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a soft ""g,"" like ""jiff""",Strongly agree,Strongly agree,Agree,Disagree,Strongly agree,Agree,Agree,Disagree,Somewhat agree,Disagree,Strongly agree,Strongly agree,Strongly disagree,Agree,Agree,Disagree,Agree,"I'm not actively looking, but I am open to new...",0.0,Not applicable/ never,Very important,Very important,Important,Very important,Very important,Very important,Important,Very important,Very important,Very important,Very important,Very important,Somewhat important,Not very important,Somewhat important,Stock options; Vacation/days off; Remote options,Yes,Other,NaN,NaN,Important,Important,Important,Somewhat important,Important,Not very important,Not very important,Not at all important,Somewhat important,Very important,NaN,NaN,Tabs,NaN,Online course; Open source contributions,NaN,NaN,NaN,6:00 AM,Swift,Swift,NaN,NaN,NaN,NaN,iOS,iOS,Atom; Xcode,Turn on some music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somewhat satisfied,Not very satisfied,Not at all satisfied,Very satisfied,Satisfied,Not very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have created a CV or Developer Story on Stac...,9.0,Desktop; iOS app,At leas

### Calculate ratio of nan values in ConvertedSalary column

In [6]:
salary_nan_ratio = df.Salary.isnull().sum()/df.shape[0]
print("NAN ratio of Salary column is ", salary_nan_ratio)
df.dropna(axis=0, subset='Salary').shape

NAN ratio of Salary column is  0.7491632938978829


(12891, 154)

### Multiple values in cells
#### Cells of some columns, which are categorical datatypes, have multiple values and are separated by semi colons(;). In this case, when performing one hot encoding using get_dummies function, a value of 1 should be given to all the corresponding columns, not just one column.

In [10]:
df.EducationTypes[0:5]

0             Online course; Open source contributions
1    Online course; Self-taught; Hackathon; Open so...
2    Self-taught; Coding competition; Hackathon; Op...
3                                                  NaN
4                                                  NaN
Name: EducationTypes, dtype: object

### I also check rows with no nan value, but there were only 6 out of 98855 rows

In [8]:
df.dropna(axis=0).shape

(0, 154)

# 3. Data Preparation

## Drop columns/rows and convert data type

### Drop row if value of ConvertedSalary column is nan

In [11]:
df = df.dropna(axis=0, subset='Salary')
df.shape

(12891, 154)

## Change datatype from string to numeric

### Change datatype of [StackOverflowRecommend, StackOverflowJobsRecommend] columns(string -> numeric)

In [83]:
df.StackOverflowRecommend = df.StackOverflowRecommend.apply(lambda value: int(value.split(' ')[0])
                                                            if type(value) == str
                                                            else float('nan')
                                                           )

In [84]:
df.StackOverflowJobsRecommend = df.StackOverflowJobsRecommend.apply(lambda value: int(value.split(' ')[0])
                                                                    if type(value) == str
                                                                    else float('nan')
                                                                   )

In [14]:
for idx, _type in df.dtypes:
    print(idx, _type)

TypeError: cannot unpack non-iterable numpy.dtype[int64] object

## Change nan values to mean values, if columns datatype is integer or float

In [16]:
columns_float = df.loc[:,df.dtypes==float].columns

for col in columns_float:
    X = df[col]
    mean = X[X.notnull()].sum()/len(X)
    df[col] = X.fillna(mean)

## Change datatype of columns that has multiple values in cells(string -> list)

### Cells with multiple values

In [41]:
contain_semicolons = df.apply(lambda col: 
                                  col.str.contains(';').any()
                                  if col.dtypes==object
                                  else False)
df_contain_semicolons = df.loc[:, contain_semicolons]
df_contain_semicolons.head(3)

,DeveloperType,MobileDeveloperType,ImportantBenefits,JobProfile,EducationTypes,SelfTaughtTypes,CousinEducation,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform,IDE,Methodology,MetricAssess,StackOverflowDevices,Gender,Race
2,Other,NaN,NaN,NaN,Self-taught; Coding competition; Hackathon; Op...,Official documentation; Trade book; Textbook; ...,NaN,Java; PHP; Python,C; Python; Rust,NaN,NaN,MySQL,NaN,NaN,NaN,Sublime Text; Vim; IntelliJ,Agile; Lean; Scrum; Extreme; Pair; Kanban,Customer satisfaction; Benchmarked product per...,Desktop; iOS browser; iOS app,Male,White or of European descent
14,Embedded applications/devices developer,NaN,NaN,LinkedIn,NaN,NaN,NaN,Assembly; C; C++,Assembly; C; C++; Python,NaN,NaN,NaN,NaN,Mac OS,Mac OS,Emacs; Xcode,NaN,Customer satisfaction; On time/in budget; Mana...,Desktop; iOS browser; iOS app,Male,White or of European descent
17,Web developer; Embedded applications/devices d...,NaN,Vacation/days off; Health benefits; Expected w...,NaN,On-the-job training; Self-taught; Coding compe...,Official documentation; Stack Overflow Q&A; Other,NaN,C; Clojure; JavaScript; Matlab; Rust; SQL; Swift,Assembly; Clojure; Go; JavaScript; Rust; Swift...,React,React; .NET Core,MySQL; PostgreSQL; Oracle,NaN,Linux Desktop; Amazon Web Services (AWS),Android; iOS; Linux Desktop; Arduino; Serverle...,Emacs; Vim,NaN,NaN,Desktop; Android browser,Male,"Native American, Pacific Islander, or Indigeno..."


In [42]:
columns_with_multiple_values = df_contain_semicolons.columns
columns_with_multiple_values

Index(['DeveloperType', 'MobileDeveloperType', 'ImportantBenefits',
       'JobProfile', 'EducationTypes', 'SelfTaughtTypes', 'CousinEducation',
       'HaveWorkedLanguage', 'WantWorkLanguage', 'HaveWorkedFramework',
       'WantWorkFramework', 'HaveWorkedDatabase', 'WantWorkDatabase',
       'HaveWorkedPlatform', 'WantWorkPlatform', 'IDE', 'Methodology',
       'MetricAssess', 'StackOverflowDevices', 'Gender', 'Race'],
      dtype='object')

### Raw one hot encoding

In [43]:
dummy = pd.get_dummies(df.ImportantBenefits)
print(dummy.shape)
dummy.iloc[:3,:]

(12891, 2047)


Annual bonus  \
2              0   
14             0   
17             0   

    Annual bonus; Charitable match; Equipment; Expected work hours; Remote options  \
2                                                   0                                
14                                                  0                                
17                                                  0                                

    Annual bonus; Charitable match; Equipment; Long-term leave; Remote options  \
2                                                   0                            
14                                                  0                            
17                                                  0                            

    Annual bonus; Charitable match; Equipment; Professional development sponsorship  \
2                                                   0                                 
14                                                  0                                 
17                                                  0                                 

    Annual bonus; Charitable match; Equipment; Professional development sponsorship; Expected work hours  \
2                                                   0                                                      
14                                                  0                                                      
17                                                  0                                                      

    Annual bonus; Charitable match; Equipment; Professional development sponsorship; Remote options  \
2                                                   0                                                 
14                                                  0                                                 
17                                                  0                                                 

    Annual bonus; Charitable match; Expected work hours; Meals  \
2                                                   0            
14                                                  0            
17                                                  0            

    Annual bonus; Charitable match; Health benefits; Education sponsorship; Meals  \
2                                                   0                               
14                                                  0                               
17                                                  0                               

    Annual bonus; Charitable match; Health benefits; Education sponsorship; Remote options  \
2                                                   0                                        
14                                                  0                                        
17                                                  0                                        

    Annual bonus; Charitable match; Health benefits; Equipment; Remote options  \
2                                                   0                            
14                                                  0                            
17                                                  0                            

    Annual bonus; Charitable match; Health benefits; Expected work hours; Remote options  \
2                                                   0                                      
14                                                  0                                      
17                                                  0                                      

    Annual bonus; Charitable match; Health benefits; Long-term leave; Child/elder care  \
2                                                   0                                    
14                                                  0                                    
17                                                  0                                    

    Annual b

### One hot encoding from list

In [44]:
# def categorize_feature(single_column):
#     if single_column.name in columns_with_multiple_values:
#         single_column = single_column.apply(lambda value: list(value.split(';'))
#                            if type(value)!=float
#                            else float('nan'))

#         return pd.get_dummies(single_column.apply(pd.Series).stack(dropna=False)).sum(level=0)
#     return pd.get_dummies(single_column)

def categorize_feature(single_column):
    single_column = single_column.apply(lambda value: list(value.split(';'))
                       if type(value)!=float
                       else float('nan'))

    return pd.get_dummies(single_column.apply(pd.Series).stack(dropna=False)).sum(level=0)

In [46]:
dummy = categorize_feature(df.ImportantBenefits)
print(dummy.shape)
dummy.iloc[:3,:]

(12891, 33)


,Annual bonus,Charitable match,Child/elder care,Education sponsorship,Equipment,Expected work hours,Health benefits,Long-term leave,Meals,None of these,Other,Private office,Professional development sponsorship,Remote options,Retirement,Vacation/days off,Annual bonus,Charitable match,Child/elder care,Education sponsorship,Equipment,Expected work hours,Health benefits,Long-term leave,Meals,None of these,Other,Private office,Professional development sponsorship,Remote options,Retirement,Stock options,Vacation/days off
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### If I use one hot encoding from list, than number of dummies are much smaller than getting dummies from raw data

## In the case of a column with a list value, it takes a long time to convert, so it is converted in advance and stored in the list

In [ ]:
categorized_df_dict = {}
categorized_df_names_list = []
for col in columns_with_multiple_values:
    df_categorized = categorize_feature(df[col])
    categorized_df_dict[col] = df_categorized
    categorized_df_names_list.append(col)

# 4. Modeling

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [93]:
random_state = 42
test_size = 0.3

In [94]:
def train_model(X_train, y_train):
    lr = LinearRegression(normalize=True)
    lr.fit(X_train, y_train)
    return lr

## In this modeling section I will find answer to my questions

Q1. What is the best feature to predict the salary   

Q2. If I collect a few features that predict best and re-train, will the result be better?    

Q3. What would be the result if we used all the features?

## Q1. What is the best feature to predict the salary?

### Use each column as X, and set rank based on accuracy

In [95]:
rank1 = [] # [accuracy, column]
Xs = df.drop('Salary', axis=1)
y = df.Salary

for col in Xs.columns:
    if Xs[col].dtypes not in [int, float]:
        if col in categorized_df_names_list:
            X = categorized_df_dict[col]
        else:
            X = pd.get_dummies(Xs[col])
    else:
        X = Xs[col]
        mean = X[X.notnull()].sum()/len(X[X.notnull()])
        X = X.fillna(mean)
        X = X.values.reshape(-1,1)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, test_size=test_size)
    
    model = train_model(X_train, y_train)
    pred = model.predict(X_test)
    score = r2_score(y_test, pred)
    
    rank1.append([score, col])

In [96]:
rank1.sort(reverse=True)
rank1[:30]

[[0.07084092115309404, 'SalaryType'],
 [0.026444499598167637, 'MilitaryUS'],
 [0.021528236302509995, 'YearsCodingProf'],
 [0.02008372731082242, 'YearsCoding'],
 [0.019718241458295926, 'Age'],
 [0.010484154209062901, 'PlatformWorkedWith'],
 [0.010256933325351048, 'RaceEthnicity'],
 [0.00991374826757152, 'LanguageWorkedWith'],
 [0.009147896072521644, 'DatabaseWorkedWith'],
 [0.008519662762147107, 'DevType'],
 [0.008175669403026165, 'IDE'],
 [0.00716434138437072, 'AssessBenefits6'],
 [0.006234377780863887, 'NumberMonitors'],
 [0.006122239629439918, 'Student'],
 [0.005837087123662932, 'AssessBenefits2'],
 [0.005156442876432021, 'CommunicationTools'],
 [0.005152791772197518, 'LastNewJob'],
 [0.004574122324387364, 'CompanySize'],
 [0.0041721109901016185, 'Methodology'],
 [0.004042567833814625, 'DatabaseDesireNextYear'],
 [0.003877247856659327, 'EducationTypes'],
 [0.0038665549020516776, 'CareerSatisfaction'],
 [0.0038515136189415378, 'Employment'],
 [0.0037903432704221585, 'LanguageDesireNex

## Q2. If I collect a few features that predict best and re-train, will the result be better?

In [35]:
num_features = [3,5,10,20,30]

In [36]:
def get_X(features):
    X = df[features]
    for column in X.columns:
        X_col = df[column]
        if X_col.dtypes not in [int, float]:
            if column in categorized_df_names_list:
                X_dummy = categorized_df_dict[column]
            else:
                X_dummy = categorize_feature(X_col)
                
            X.drop(column, axis=1, inplace=True)    
            X = pd.concat([X, X_dummy], axis=1)
        else:            
            mean = X_col[X_col.notnull()].sum()/len(X_col)
            X[column] = X[column].fillna(mean)

    return X

In [37]:
rank2 = [] # [accuracy, number of columns]
for num in num_features:   
    features = []
    for i in range(num):
        features.append(rank1[i][1])
    
    X = get_X(features)
    y = df.Salary
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, test_size=test_size)
    
    model = train_model(X_train, y_train)
    pred = model.predict(X_test)
    score = r2_score(y_test, pred)
    
    print(f"Score for {num} features are {score}")
    rank2.append([score, num])

Score for 3 features are 0.1457982443211049
Score for 5 features are 0.1492681415025816
Score for 10 features are 0.16409562527544763
Score for 20 features are 0.1686434585622112
Score for 30 features are 0.16970959161988775


In [38]:
rank2.sort(reverse=True)
rank2

[[0.16970959161988775, 30],
 [0.1686434585622112, 20],
 [0.16409562527544763, 10],
 [0.1492681415025816, 5],
 [0.1457982443211049, 3]]

## Q3. What would be the result if I used all the features?

In [103]:
def get_all_X():
    features = df.drop('Salary', axis=1).columns
    
    X = df[features]
    for column in X.columns:
        X_col = df[column]
        if X_col.dtypes not in [int, float]:
            if column in categorized_df_names_list:
                X_dummy = categorized_df_dict[column]
            else:
                X_dummy = pd.get_dummies(X_col)
            X.drop(column, axis=1, inplace=True)
            X = pd.concat([X, X_dummy], axis=1)
        else:            
            mean = X_col[X_col.notnull()].sum()/len(X_col)
            X[column] = X[column].fillna(mean)

    return X

In [104]:
X = get_all_X()
y = df.Salary

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, test_size=test_size)

model = train_model(X_train, y_train)
pred = model.predict(X_test)
score = r2_score(y_test, pred)

print(f"Score: {score}")

Score: -1.3880575740019469e+24


## Additional Question. How many features do I need to have satisfactory results?

### I will use get_X() from Q2

In [31]:
total_num = df.shape[1]
num_features = list(range(3, 10))
for i in range(len(num_features)):
    num_features[i] = int(num_features[i]*0.1*total_num)   
    
rank4 = [] # [accuracy, number of columns]
for num in num_features:   
    features = []
    for i in range(num):
        features.append(rank1[i][1])
    
    X = get_X(features)
    y = df.Salary
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, test_size=test_size)
    
    model = train_model(X_train, y_train)
    pred = model.predict(X_test)
    score = r2_score(y_test, pred)
    
    print(f"Score for {num} features are {score}")
    rank2.append([score, num])

Score for 37 features are 0.1720824138136795
Score for 50 features are 0.1729736280130405
Score for 62 features are 0.17269775986155145
Score for 75 features are 0.1732827448135189
Score for 87 features are 0.17258608709146073
Score for 100 features are 0.17238779107057445


KeyboardInterrupt: 

In [99]:
features = ['OpenSource', 'CompanySize', 'YearsCoding', 'YearsCodingProf', 'JobSatisfaction', 'CareerSatisfaction','FormalEducation']
X = get_X(features)
y = df.Salary

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, test_size=test_size)

model = train_model(X_train, y_train)
pred = model.predict(X_test)
score = r2_score(y_test, pred)

print(f"Score: {score}")

Score: 0.02881547239061688


In [102]:
model = train_model(X, y)
pred = model.predict(X)
score = r2_score(y, pred)
print(f"Score: {score}")

Score: 0.031151666850835147
